<a href="https://colab.research.google.com/github/hashemi124/ANN_edu_deep/blob/master/caltech101_vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
dir = '/content/drive/My Drive/Colab Notebooks/Caltech101/train'

width = 299
height = 299

In [ ]:
data_generator = ImageDataGenerator(
    rotation_range = 10,
    horizontal_flip = True,
    vertical_flip = True,
    zoom_range = 0.1,
    width_shift_range = 0.2,
    height_shift_range = 0.1,
    rescale = 1 / 255,
    validation_split =0.1,
  
)

train_data = data_generator.flow_from_directory(
    dir,
    target_size = (width, height),
    batch_size = 8,
    # save_to_dir = '/content/drive/My Drive/Colab Notebooks/augment_animals',
    class_mode = 'categorical',
    subset = 'training'
)

validation_data = data_generator.flow_from_directory(
   dir,# '/content/drive/My Drive/Colab Notebooks/Caltech101/test',
    target_size = (width, height),
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation'
)

In [4]:
base_model = tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    #classes=15,
    classifier_activation="softmax",
)

In [ ]:
base_model.summary()

In [6]:
model = tf.keras.models.Sequential([

    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(64, activation='relu'),
    Dense(15, activation='softmax'),
])



In [ ]:
model.summary()

In [ ]:

tf.keras.utils.plot_model(model, show_shapes=True)

In [8]:
model.compile(optimizer=SGD(), 
              loss = tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [9]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]

In [ ]:
batch_size = 8

output = model.fit(
    train_data,
    steps_per_epoch = train_data.samples // batch_size,
    epochs = 100,
    validation_data = validation_data, 
    validation_steps = validation_data.samples // batch_size,    
   # class_weight = {0:1 , 1:1 , 2:2},
    shuffle = True,
    #callbacks=callbacks
)

Epoch 1/100
50/50 [==============================] - 1517s 30s/step - loss: 2.6436 - accuracy: 0.1335 - val_loss: 2.5555 - val_accuracy: 0.2000
Epoch 2/100
29/50 [================>.............] - ETA: 10:14 - loss: 2.3483 - accuracy: 0.2707

In [ ]:
import cv2 as cv
img = cv.imread('image.png')
img = cv.resize(img, (299, 299))
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

img = img / 255.0

img = img.reshape(1, 299, 299, 3)

y_pred = model.predict(img)
print(y_pred)
print(np.argmax(y_pred))